In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error


In [3]:
!pip install nba_api

from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog, playergamelog,leaguegamefinder
from nba_api.live.nba.endpoints import boxscore
from nba_api.stats.endpoints import boxscoretraditionalv2
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playerfantasyprofile
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import TeamYearByYearStats

  Using cached nba_api-1.4.1-py3-none-any.whl (261 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.0 MB/s eta 0:00:00
  Using cached certifi-2023.11.17-py3-none-any.whl (162 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 19.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.9.24
    Uninstalling certifi-2022.9.24:
      Successfully uninstalled certifi-2022.9.24
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not in

In [4]:
teams_df = pd.DataFrame(teams.get_teams())
players_df = pd.DataFrame(players.get_players())
Lebron_id=2544
Giannis_id=203507
player_fantasy_data = playerfantasyprofile.PlayerFantasyProfile(player_id=Lebron_id)
player_fantasy_data = player_fantasy_data.get_data_frames()

In [11]:
last_5_games=player_fantasy_data[2]
vs_opp_stats=player_fantasy_data[4]
print(vs_opp_stats.GROUP_SET[0])

vs. Opponent


In [12]:
last_5_games_stats = last_5_games[last_5_games['GROUP_VALUE'] == 'Last 5 Games']
print(last_5_games_stats)

      GROUP_SET   GROUP_VALUE  GP  W  L  W_PCT         MIN  FGM  FGA  FG_PCT  \
0  Last N Games  Last 5 Games   5  4  1    0.8  173.613333   56   87   0.644   

   ...  BLK  BLKA  PF  PFD  PTS  PLUS_MINUS  DD2  TD3  FAN_DUEL_PTS  \
0  ...    1     5   3   27  145          48    2    0         246.0   

   NBA_FANTASY_PTS  
0            253.0  

[1 rows x 32 columns]


In [19]:
gamelog = playergamelog.PlayerGameLog(player_id=Lebron_id, season=2023)
gamelog_data = gamelog.get_data_frames()
vs_warriors = gamelog_data[0][(gamelog_data[0]['MATCHUP'] == 'LAL vs. GSW') | (gamelog_data[0]['MATCHUP'] == 'LAL @ GSW')]
print(vs_warriors)
team_log = teamgamelog.TeamGameLog(team_id=1610612744)
team_log_data = team_log.get_data_frames()[0]
# Filter the last 5 games
team_last_5_games = team_log_data.head(5)
print(team_last_5_games)

   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0      22023       2544  0022301155  APR 09, 2024  LAL vs. GSW  L   36   14   
10     22023       2544  0022300973  MAR 16, 2024  LAL vs. GSW  L   38   15   
27     22023       2544  0022300650  JAN 27, 2024    LAL @ GSW  W   48   14   

    FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0    22   0.636  ...     6    7   11    2    0    4   0   33          -6   
10   23   0.652  ...     5    8    9    1    1    4   1   40          -6   
27   25   0.560  ...    19   20   12    0    0    3   2   36          -6   

    VIDEO_AVAILABLE  
0                 1  
10                1  
27                1  

[3 rows x 27 columns]
      Team_ID     Game_ID     GAME_DATE      MATCHUP WL   W   L  W_PCT  MIN  \
0  1610612744  0022301169  APR 11, 2024    GSW @ POR  W  45  35  0.563  240   
1  1610612744  0022301155  APR 09, 2024    GSW @ LAL  W  44  35  0.557  240   
2  1610612744  0022301142  APR

In [21]:
Lebron_id = 2544
gamelog = playergamelog.PlayerGameLog(player_id=Lebron_id, season='2023')
lebron_gamelog = gamelog.get_data_frames()[0]
print(lebron_gamelog)

   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0      22023       2544  0022301155  APR 09, 2024  LAL vs. GSW  L   36   14   
1      22023       2544  0022301127  APR 06, 2024  LAL vs. CLE  W   36   10   
2      22023       2544  0022301103  APR 03, 2024    LAL @ WAS  W   36    9   
3      22023       2544  0022301094  APR 02, 2024    LAL @ TOR  W   29   10   
4      22023       2544  0022301078  MAR 31, 2024    LAL @ BKN  W   37   13   
..       ...        ...         ...           ...          ... ..  ...  ...   
64     22023       2544  0022300127  NOV 01, 2023  LAL vs. LAC  W   42   13   
65     22023       2544  0022300111  OCT 30, 2023  LAL vs. ORL  W   33    7   
66     22023       2544  0022300100  OCT 29, 2023    LAL @ SAC  L   39   11   
67     22023       2544  0022300076  OCT 26, 2023  LAL vs. PHX  W   35    7   
68     22023       2544  0022300061  OCT 24, 2023    LAL @ DEN  L   29   10   

    FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV

In [22]:
merged_data = pd.merge(lebron_gamelog, team_log_data, on='Game_ID', suffixes=('_LeBron', '_Warriors'))
print(merged_data.columns)

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE_LeBron',
       'MATCHUP_LeBron', 'WL_LeBron', 'MIN_LeBron', 'FGM_LeBron', 'FGA_LeBron',
       'FG_PCT_LeBron', 'FG3M_LeBron', 'FG3A_LeBron', 'FG3_PCT_LeBron',
       'FTM_LeBron', 'FTA_LeBron', 'FT_PCT_LeBron', 'OREB_LeBron',
       'DREB_LeBron', 'REB_LeBron', 'AST_LeBron', 'STL_LeBron', 'BLK_LeBron',
       'TOV_LeBron', 'PF_LeBron', 'PTS_LeBron', 'PLUS_MINUS',
       'VIDEO_AVAILABLE', 'Team_ID', 'GAME_DATE_Warriors', 'MATCHUP_Warriors',
       'WL_Warriors', 'W', 'L', 'W_PCT', 'MIN_Warriors', 'FGM_Warriors',
       'FGA_Warriors', 'FG_PCT_Warriors', 'FG3M_Warriors', 'FG3A_Warriors',
       'FG3_PCT_Warriors', 'FTM_Warriors', 'FTA_Warriors', 'FT_PCT_Warriors',
       'OREB_Warriors', 'DREB_Warriors', 'REB_Warriors', 'AST_Warriors',
       'STL_Warriors', 'BLK_Warriors', 'TOV_Warriors', 'PF_Warriors',
       'PTS_Warriors'],
      dtype='object')


In [ ]:
features = ['PTS_LeBron', 'AST_LeBron', 'REB_LeBron', 'PTS_Warriors']
X = merged_data[features]
y = merged_data['PTS_LeBron']  # Predict LeBron's points

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

In [ ]:
latest_lebron_stats = lebron_gamelog.iloc[-1][['PTS', 'AST', 'REB']]

# Extract the latest game's stats for the Warriors
latest_warriors_stats = team_last_5_games.iloc[-1][['PTS']]

# Combine LeBron's and the Warriors' stats into a single DataFrame
prediction_data = pd.concat([latest_lebron_stats, latest_warriors_stats])

In [ ]:
print("---------------------------------------------")
print(f"\n\n")
print("---------------------------------------------")
print(prediction_data)
print(np.shape(prediction_data))
print("---------------------------------------------")
print(f"\n\n")
print("---------------------------------------------")

In [ ]:
X_pred = prediction_data.values.reshape(1, -1)

# Make predictions using the trained model
predicted_points = rf_model.predict(X_pred)

# Print the predicted points
# print("Predicted points for LeBron's next game against the Warriors:", predicted_points[0])

# Check accuracy using regression metrics (R^2)
print(f"\n\nTrue values: {[y_test[0]]}")
print(f"\n\nPredicted values: {[predicted_points[0]]}")
score = r2_score([y_test[0]], [predicted_points[0]])
print("\n\nThe accuracy of our model is {}%".format(round(score, 2) * 100))
